In [1]:
# imports
import os
import cv2
from sklearn.ensemble import BaggingClassifier
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#import sklearn as skl
import sklearn.preprocessing as preproc
import numpy as np
import keras as k
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D, Flatten, Dense, Dropout, MaxPool2D
#import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
#from time import time
import pandas as pd
from imblearn.over_sampling import SMOTENC
import tensorflow as tf
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
import math
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
#### variabili globali
#### GIUSEPPE
image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
#### FABIO
# image_path = '..\skin-cancer-mnist-ham10000\HAM10000_images_part_1'
csv_path = 'HAM10000_metadata.csv'
csv_completo = 'dataframe_completo.csv'

In [27]:
######## API varie
def lab_encode(dataset, le):#Ho una sola colonna! 
    new_ds = dataset
    new_ds["dx"] = le.fit_transform(dataset["dx"])  
    return new_ds

def lab_decode(dataset, le):
    new_ds = dataset
    new_ds["dx"] = le.inverse_transform(dataset["dx"])  
    return new_ds

# conteggio statistiche dataset
def stats(dataset):
    labels = set(dataset['dx'])
    for label in labels:
        print(str(label), ':\t', str(dataset[dataset['dx']==label]['dx'].count()))
        #if label != "nv":
        #    x = dataset[dataset['dx']==label]['dx'].count()
        #    y = dataset[dataset['dx']=="nv"]['dx'].count()
        #    print("ugmented per image: ", str(math.floor((y-x)/x)))
        x = dataset[dataset['dx']==label]['dx'].count()
        y = dataset[dataset['dx']==5]['dx'].count()
        value = math.floor((y-x)/x)
        print("ugmented per image: ", str(value))
        aug_size[label] = value
        #print (label)
        
def new_classifier1():
    input_img = Input(shape=(450, 600, 3))  # 3x600x450 image RGB 
    #print (tf.shape(input_img))
    x = Conv2D(20, (5, 5), activation='relu', padding='same')(input_img) # 20x450x600
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 20x225x300
    #print (tf.shape(x))
    x = Conv2D(140, (3, 3), activation='relu', padding='same')(x) # 140x75x100
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 40x38x50
    #print (tf.shape(x))
    x = Conv2D(50, (3, 3), activation='relu', padding='same')(x) # 50x38x50
    #print (tf.shape(x))
    x = MaxPooling2D((2, 2), padding='same')(x) # 50x19x25
    #print (tf.shape(x))
    x = Flatten()(x) # qui la x diventa monodimensionale
    #print (tf.shape(x))
    # da qualche parte ci va il numero delle label?
    encoded = Dense(7, activation='softmax')(x)
    classifier = k.models.Model(input_img, encoded)   #questo è il nostro base_estimator, compile configura per il training
    #classifier.compile(optimizer='adadelta', loss='binary_crossentropy')
    classifier.compile(optimizer='adadelta', loss='categorical_crossentropy') # <- questo potrebbe risolvere
    return classifier

def new_classifier2():
    # Set the CNN model 
    input_shape = (450, 600, 3)
    num_classes = 7
    
    model = k.models.Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5),activation='relu',padding = 'Same', input_shape = input_shape))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(140,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(50,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

def new_classifier3():
    # Set the CNN model 
    input_shape = (150, 200, 3)
    num_classes = 7
    
    model = k.models.Sequential()
    model.add(Conv2D(20, kernel_size=(5, 5),activation='relu',padding = 'Same', input_shape = input_shape))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(140,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Conv2D(50,kernel_size=(3, 3), activation='relu',padding = 'Same'))
    model.add(MaxPool2D(pool_size = (2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    model.compile(optimizer='adadelta', loss='binary_crossentropy')
    return model

def initialize_dataset():
    dataset = pd.read_csv(csv_path, encoding = "ISO-8859-1")
    labels = set(dataset["dx"])
    new_ds = pd.DataFrame(dataset)
    columns=["lesion_id", "dx_type", "age", "sex", "localization"]
    new_ds = new_ds.drop(columns=columns, axis=0)
    return new_ds, labels

# dal dataframe ne estrae le immagini in un array, serve nel batch
def load_images(array):
    images = list()
    #print(array.iloc[0])
    for index in range(0,len(array)):
        #print(index.type())
        img_id = array.iloc[index]
        elem = os.path.join("..\skin-cancer-mnist-ham10000\HAM10000_images_part_1", img_id)
        elem = elem + ".jpg"
        img = cv2.imread(elem)
        # TODO va fatto un reshape, non capisco perché
        images.append(img)
    return images

In [ ]:
############################ INIZIALIZZAZIONE
#creazione dataframe
ds, labels = initialize_dataset()
# encoding delle labels (dx)
encoder = preproc.LabelEncoder()
encoded = lab_encode(ds, encoder)
encoded = encoded.sort_index()

aug_size = [0,0,0,0,0,0,0]

# Create a data generator
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    #brightness_range=(0.9,1.1),
    fill_mode='nearest')

In [ ]:
stats(encoded)

In [ ]:
##### creazione augmented

# note that we are not augmenting class 'nv'
#class_list = ['0','1','2','3','4','6']
class_list = ['3','4','6']

aug_dir = '..\\aug_dir'
img_dir = os.path.join(aug_dir, 'img_dir')    


# Choose a class
for img_class in class_list:
    
    # creazione cartelle contenenti le augmented
    dst_class_dir = os.path.join(img_dir, img_class)
    os.mkdir(dst_class_dir)
    
    
    # list all images in that directory
    src_class_dir = os.path.join('..\\skin-cancer-mnist-ham10000\\HAM10000_images_part_1', img_class)
    img_list = os.listdir(src_class_dir) # qui le immagini sono già nelle proprie cartelle
    print('#img: '+ str(len(img_list)))
    
    # per ogni immagine devo creare le augmented
    for fname in img_list:
        fpath = os.path.join(src_class_dir, fname)
        img = Image.open(fpath)  # this is a PIL image
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        
        prefix = fname.replace(".jpg", "")
        # the .flow() command below generates batches of randomly transformed images
        # and saves the results to the `preview/` directory
        i = 0
        for batch in datagen.flow(x, batch_size=1,
                                  save_to_dir=dst_class_dir, save_prefix=prefix , save_format='jpg'):
            i += 1
            if i > aug_size[int(img_class)]:
                break  # otherwise the generator would loop indefinitely
        
        del x
        img.close()
    # run the generator and create about 6000 augmented images
    #for i in range(0,num_batches):
    #
    #    imgs, labels = next(aug_datagen)

In [ ]:
############## sistemazione nuovo dataset
aug_dir = '..\\aug_dir'
img_dir = os.path.join(aug_dir, 'img_dir') 
classes = os.listdir(img_dir)

columns = encoded.columns
tupla = pd.DataFrame(columns=columns)
index = 10015

for elem1 in classes:
    class_dir = os.path.join(img_dir, elem1)
    images = os.listdir(class_dir)
    for elem2 in images:
        elem2 = elem2.replace('.jpg', '')
        tupla = tupla.append({'image_id': elem2, 'dx': int(elem1)}, ignore_index=True)

In [ ]:
# salvataggio (punto 1.2)
#dataset.to_csv('Pokemon_clean.csv', index=False) 
encoded = encoded.append(tupla)
encoded.to_csv('dataframe_completo.csv', index=False) 

In [ ]:
encoded

In [ ]:
from sklearn.model_selection import train_test_split

# da provare
X = encoded["pixels"]
y = pd.get_dummies(encoded["dx"])
#bag_class = ensemble()
X = np.asarray(X.tolist())
y = np.asarray(y)

# splitting test and validation
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.30)
X_test, X_val, y_test, y_val = train_test_split(X_test1, y_test1, test_size=0.40)


classifier = new_classifier3()
# STUDIARE BENE IL FIT
classifier.fit(X_train, y_train, epochs=1, batch_size=100, shuffle=True, validation_data=(X_val, y_val)) 

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train)

cl2 = new_classifier3()
# fits the model on batches with real-time data augmentation:
cl2.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32, epochs=1)

# here's a more "manual" example
for e in range(2):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(X_train, y_train, batch_size=20):
        cl2.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(X_train) / 20:
            print('fine batch ', batches)
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

In [81]:
### inizializzazione classificatore sul disco
cl1 = new_classifier1()
cl1.save('classifier1.h5')

In [4]:
################### TRAINING
#- copia temporanea dataset completo tmp
#- dichiarazione size per il batch
#- fin quando dim(tmp) >= size:
#    - prendo size campioni random da tmp
#    - train_test_split dei campioni
#    - se esiste carico il modello del classificatore
#    - fit del modello con questi campioni:
#        - carico le immagini
#        - converto in array
#        - fit()                                                              (DA STUDIARE BENE I PARAMETRI, https://keras.io/models/sequential/)
#        - dealloco le immagini
#    - salvo il modello (dovrebbe funzionare perché non lo dealloco mai)
#    - cancello i campioni da tmp
#    - aggiorno dim
#    - dealloco tutto

dataset = pd.read_csv(csv_completo, encoding = "ISO-8859-1")
batch = 1000


def my_train_batch(dataset=dataset, batch=500, classifier='classifier1.h5'):
    size = len(dataset)
    epoch=0
    while size >= batch:
        # To get n random rows 
        samples = dataset.sample(n = batch)
        # splitting test and validation
        X_train, X_test1, y_train, y_test1 = train_test_split(samples["image_id"], samples["dx"], test_size=0.30)
        X_test, X_val, y_test, y_val = train_test_split(X_test1, y_test1, test_size=0.40)
        del X_test1, y_test1
        train_img = load_images(X_train)
        test_img = load_images(X_test)  # NB non sono mai usate, pensare se servano oppure no
        val_img = load_images(X_val)
        cl1 = k.models.load_model('classifier1.h5')
        cl1.fit(np.asarray(train_img), y_train, epochs=1, shuffle=True, validation_data=(np.asarray(val_img), y_val))
        cl1.save('classifier1.h5')
        del X_train, y_train, X_test, y_test, X_val, y_val, train_img, test_img, val_img, classifier
        dataset = dataset.drop(samples.index)
        size = len(dataset)
        del samples
        epoch = epoch + 1
        print("fine epoca " + epoch + ";\trestano " + size + "campioni nel dataset")
    return

In [28]:
samples = dataset.sample(n=2000)
my_train_batch(dataset=samples, batch=batch)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 700 arrays: [array([[[157, 134, 216],
        [154, 131, 215],
        [153, 134, 215],
        ...,
        [162, 145, 212],
        [161, 147, 211],
        [162, 149, 217]],

       [[157, 136, 215],
        [...